In [263]:
import random
import numpy as np
import matplotlib as plt
from datetime import timedelta, date, datetime
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd


day_contagious = []
patients = []
time = []
coordinate = []

class Patient:

    def __init__(self, first_name, last_name, health):
        self.first_name = first_name
        self.last_name = last_name
        self.health = health
        self.friends = []
        self.location = []

    def add_friend(self, friend_person):
        self.friends.append(friend_person)

    def add_location(self, location):
        self.location = location

    def get_friends(self):
        return self.friends
    
    def get_location(self):
        return self.location
    
    def get_name(self):
        return self.first_name + self.last_name

    def get_health(self):
        return self.health

    def infect(self, viral_load):
        if self.health <= 29:
            self.health -= 0.1 * viral_load

        elif 29 < self.health < 50:
            self.health -= viral_load

        elif self.health >= 50:
            self.health -= 2 * viral_load

        if self.health < 0:
            self.health = 0

    def is_contagious(self):
        if round(self.health, 0) < 50:
            return True
        else:
            return False

    def sleep(self):
        if self.health < 96:
            self.health += 5
        else:
            self.health = 100

    def set_health(self, new_health):
        self.health = new_health
    
    def set_location(self, location):
        self.location = location
    
    def __str__(self):
        return self.first_name + " " + self.last_name + " " + str(self.health) + " " + self.health

    
def collect_data(start_date, end_date):
    for patient in patients:
        position = patient.get_location()
        coordinate.append(position)
        
    
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

        
def load_patients(initial_health):
    f = open("/Users/MichaelFu/PycharmProjects/FIT9136/a2_sample_set2.txt", "r")
    f1 = f.readlines()
    global patients
    patients = []

    for line in f1:
        line_split_list = line.split(":")
        patient_split_list = line_split_list[0].split()
        one_patient = Patient(patient_split_list[0], patient_split_list[1], initial_health)
        patients.append(one_patient)

    cur_index = 0
    for line in f1:
        line_split_list = line.split(":")
        all_patient_split_list = line_split_list[1].split(",")

        for patient_name in all_patient_split_list:
            single_patient_split_list = patient_name.split()
            name = single_patient_split_list[0].strip() + single_patient_split_list[1].strip()

            for patient in patients:
                if patient.get_name() == name:
                    patients[cur_index].add_friend(patient)
                    break
        cur_index += 1
    f.close()


def locate_patients():
    for patient in patients:
        south_boundary = -37.9656
        north_boundary = -37.8726
        east_boundary = 145.2330
        west_boundary = 145.0247
        location = [random.uniform(west_boundary, east_boundary), random.uniform(south_boundary, north_boundary)]
        patient.add_location(location)


def run_simulation(start_date, end_date, meeting_probability, patient_zero_health):  
    global coordinate
    total_contagious = []
    first_patient = patients[0]
    first_patient.set_health(patient_zero_health)
    
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime('%Y-%m-%d')
        print(date)
        day_contagious = 0
        for man in patients:
            viral_load = 5 + ((man.get_health() - 25) ** 2) / 62
            friends = man.get_friends()
            for friend in friends:
                viral_load_friend = 5 + ((friend.get_health() - 25) ** 2) / 62
                r = random.randint(1, 100) / 100
                if meeting_probability >= r and man.is_contagious():
                    friend.infect(viral_load)
                if meeting_probability >= r and friend.is_contagious():
                    man.infect(viral_load_friend)
       
        for man in patients:
            if man.is_contagious():
                day_contagious += 1
                coordinate.append(man.get_location())
                time.append(date)
                
            man.sleep()
        
        total_contagious.append(day_contagious)
    
    return total_contagious

coordinate = []
day_contagious = []
patients = []
time = []
load_patients(99)
locate_patients()
start_date = date(2020, 2, 1)
end_date = date(2020, 2, 10)
run_simulation(start_date, end_date, 0.8, 49)

m = folium.Map(location = [-37.9145, 145.1275], zoom_start = 12)

multipoint_1 = [{
    'type': 'Feature',
    'geometry': {
        'type': 'MultiPoint',
        'coordinates': coordinate,
    },
    'properties': {
        'style': {
            'color': 'blue',
            'popup' : 'contagious'
        },
        'times': time,
    }
}]

plugins.TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': multipoint_1},
    period='P1D',
    duration='P1M',
    auto_play=False,
    loop=False,
    loop_button=True,
    date_options='YYYY/MM/DD',
).add_to(m)
m

2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10


In [260]:
import folium
from folium import plugins
m = folium.Map(
    location = [-37.9145, 145.1275],
    zoom_start = 12
)

plugins.TimestampedGeoJson({
    'type': 'Feature',
    'geometry': {
    'type': 'LineString',
    'coordinates': ['2019-09-01','2019-09-02','2019-09-03'],
    },
    'properties': {
    'times': [1435708800000, 1435795200000, 1435881600000]
    }
}, period='PT1D', add_last_point=True).add_to(m)

display(m)

In [261]:
import json
l = [12,"b",'c']
print(type(l))
j = json.dumps(l)
print(type(j))

<class 'list'>
<class 'str'>


In [262]:
import folium
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[52.467697, -2.548828], zoom_start=6)

polygon_1 = {
    'type': 'Feature',
    'geometry': {
        'type': 'MultiPoint',
        'coordinates': [
             [-2.548825, 51.467690],
             [-0.087891, 51.536086],
             [-37.1111, 145.1275],
        ],
    },
    'properties': {
        'style': {
            'color': 'red',
        },
        'times': ['2015-07-22T00:00:00', '2015-07-22T00:00:00', '2015-07-23T00:00:00']
    }
}

TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': [polygon_1]},
    period='P1D',
    duration='P1D',
    auto_play=False,
    loop=False,
    loop_button=True,
    date_options='YYYY/MM/DD',
).add_to(m)
    
m

In [144]:
"""
m = folium.Map(
location = [-37.9145, 145.1275],
zoom_start = 12)
for patient in patients:
    patient_location = patient.get_location()
    if patient.is_contagious():     
        folium.Marker(
        location = patient_location,
        popup = 'Patient_contagious',
        icon = folium.Icon(icon='cloud', color = 'red')).add_to(m)

    else:
        folium.Marker(
        location = patient_location,
        popup = 'Normal_person',
        icon = folium.Icon(icon='cloud', color = 'blue')).add_to(m) 
m
"""

"\nm = folium.Map(\nlocation = [-37.9145, 145.1275],\nzoom_start = 12)\nfor patient in patients:\n    patient_location = patient.get_location()\n    if patient.is_contagious():     \n        folium.Marker(\n        location = patient_location,\n        popup = 'Patient_contagious',\n        icon = folium.Icon(icon='cloud', color = 'red')).add_to(m)\n\n    else:\n        folium.Marker(\n        location = patient_location,\n        popup = 'Normal_person',\n        icon = folium.Icon(icon='cloud', color = 'blue')).add_to(m) \nm\n"

In [147]:
map = folium.Map(location=[-37.9145, 145.1275], zoom_start=12 , 
                 tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
                 attr = "IGN")
map


In [234]:
"""
position = [[]]
lat = []
long = []
name = []
health = []
for patient in patients:
    name.append(patient.get_name())
    health.append(get_health())
    position.append(patient.get_location)
for p in position
    lat = p[0]
    long = p[1]
    
    
data = {'Full_Name': name,
        'Health_Point': health,
        'Lat': lat,
        'Long': long      
       }
df = pd.DataFrame (data, columns = ['First Column Name','Second Column Name',...])
"""

"\nposition = [[]]\nlat = []\nlong = []\nname = []\nhealth = []\nfor patient in patients:\n    name.append(patient.get_name())\n    health.append(get_health())\n    position.append(patient.get_location)\nfor p in position\n    lat = p[0]\n    long = p[1]\n    \n    \ndata = {'Full_Name': name,\n        'Health_Point': health,\n        'Lat': lat,\n        'Long': long      \n       }\ndf = pd.DataFrame (data, columns = ['First Column Name','Second Column Name',...])\n"

In [235]:
points = [
{
    'time': '2019-09-01',
    'popup': '<h1>address1</h1>',
    'coordinates': [18.528387, 73.874251]
},
{
    'time': '2019-09-02',
    'popup': '<h1>address1</h1>',
    'coordinates': [18.456863, 73.801601]
},
{
    'time': '2019-09-03',
    'popup':'<h1>address1</h1>',
    'coordinates': [18.527615, 73.872384]
},
{
    'time': '2019-09-04',
    'popup': '<h1>address1</h1>',
    'coordinates': [18.528387, 73.874251]},
{
    'time': '2019-09-05',
    'popup': '<h1>address1</h1>',
    'coordinates': [18.456863, 73.801601]}]
features = [
{
    'type': 'Feature',
    'geometry': {
        'type': 'Point',
        'coordinates': point['coordinates'],
    },
    'properties': {
        'time': point['time'],
        'popup': point['popup']
    }
} for point in points]

features.append(
{
    'type': 'Feature',
    'geometry': {
        'type': 'LineString',
        'coordinates':[
            [18.528387, 73.874251],
            [18.456863, 73.801601],
            [18.527615, 73.872384],
            [18.528387, 73.874251],
            [18.456863, 73.801601]
        ] ,
    },
    'properties': {
        'popup': 'Current address',
        'times': [
            '2019-09-01',
            '2019-09-02',
            '2019-09-03',
            '2019-09-04',
            '2019-09-05'
        ]
    }
})
m = folium.Map(
location=[18.5204,73.8567],
tiles='cartodbpositron',
zoom_start=10,)

plugins.TimestampedGeoJson(
{
    'type': 'FeatureCollection',
    'features': features
},
auto_play=False,
loop=False,
#max_speed=1,
loop_button=True,
date_options='YYYY/MM/DD',
#time_slider_drag_update=True,
duration='P2D').add_to(m)
m